# 如何设计&开发新的明密文混合算法

> This tutorial is only available in Chinese.

> 以下代码仅作为示例，请勿在生产环境直接使用。

## 集群部署模式

隐语有多种集群部署模式，包括[仿真模式](../getting_started/deployment.md#simulation)、[生产模式](../getting_started/deployment.md#production)和[调试模式](../getting_started/debug_mode.md)。不同的集群部署模式对代码编写几乎没有影响，只有集群初始化的代码有所不同。因此本教程中的示例都是以仿真模式编写和运行的。

下面首先以单机仿真模式启动一个4节点的集群：

In [1]:
import secretflow as sf

# In case you have a running SecretFlow runtime already.
sf.shutdown()

sf.init(['alice', 'bob', 'carol', 'dave'], address='local')

2023-11-06 20:03:44,807	INFO worker.py:1538 -- Started a local Ray instance.


## Device 概念 

在开始前建议先阅读 SecretFlow 的[架构设计](../developer/design/architecture.md)。

在集群初始化时，我们声明了4个参与方：alice、bob、carol、dave，它们对应4个“物理设备”。在 SecretFlow 中物理设备用来直观的表示参与方和组网信息，用于实际执行计算和网络通信。

隐语在物理设备上构建了一套逻辑设备：PYU、SPU、HEU、TEEU。一个逻辑设备由一个或多个物理设备组成，同时一个物理设备也可以用于多个逻辑设备。在编程意义上，逻辑设备负责管理“数据”和执行“计算”，在隐语上编程可以简单理解为 3 个部分：
1. 像明文编程一样编写逻辑设备的计算过程；
2. 将数据放到逻辑设备上，或者在逻辑设备之间流转；
3. 控制逻辑设备，使用该逻辑设备上的“数据”执行一段计算。

可以说隐语上的编程就是面向逻辑设备的编程。

In [2]:
# 声明两个 PYU 逻辑设备，alice 和 bob，分别代表在对应的物理设备本地计算
alice, bob = sf.PYU('alice'), sf.PYU('bob')

alice, bob, object.__repr__(alice)

(PYURuntime(alice),
 PYURuntime(bob),
 '<secretflow.device.device.pyu.PYU object at 0x7f4a9b8c0e80>')

In [3]:
# 使用两个物理设备组成一个 SPU 逻辑设备
spu = sf.SPU(sf.utils.testing.cluster_def(['carol', 'dave']))

spu

(SPURuntime pid=3915774) 2023-11-06 20:05:54.079 [info] [default_brpc_retry_policy.cc:DoRetry:52] socket error, sleep=1000000us and retry
(SPURuntime pid=3915774) 2023-11-06 20:05:55.079 [info] [default_brpc_retry_policy.cc:LogHttpDetail:29] cntl ErrorCode '112', http status code '200', response header '', error msg '[E111]Fail to connect Socket{id=0 addr=127.0.0.1:54955} (0x0x335c640): Connection refused [R1][E112]Not connected to 127.0.0.1:54955 yet, server_id=0'
(SPURuntime pid=3915774) 2023-11-06 20:05:55.079 [info] [default_brpc_retry_policy.cc:DoRetry:75] aggressive retry, sleep=1000000us and retry
(SPURuntime pid=3915774) 2023-11-06 20:05:56.079 [info] [default_brpc_retry_policy.cc:LogHttpDetail:29] cntl ErrorCode '112', http status code '200', response header '', error msg '[E111]Fail to connect Socket{id=0 addr=127.0.0.1:54955} (0x0x335c640): Connection refused [R1][E112]Not connected to 127.0.0.1:54955 yet, server_id=0 [R2][E112]Not connected to 127.0.0.1:54955 yet, server_id

## 在 PYU 明文设备上编程

明文设备和物理设备对应，用于编写和运行在单方明文执行的代码。
下面以一个多方求均值的例子演示一下如何在 PYU 设备上编写代码。

任务：alice、bob、carol 三方各自有一些整型数值， 目标是计算出三方所有数字的均值，dave 是一个可用的辅助计算方。

### 声明 PYU 设备 

In [4]:
alice = sf.PYU('alice')
bob = sf.PYU('bob')
carol = sf.PYU('carol')
dave = sf.PYU('dave')

### 准备数据

In [5]:
import numpy as np


# 定义数据生成函数
def gen_data(fake_seed, file_name):
    import pathlib

    pathlib.Path(file_name).parent.mkdir(parents=True, exist_ok=True)

    data = np.arange(fake_seed, 2 * fake_seed - 1, 1)
    np.savetxt(file_name, data, fmt='%d')

三方分别执行 `gen_data`，并把生成的数据保存的各自的本地，由于目前是单机仿真模式，所以实际上都会保存在本机上。

In [6]:
alice(gen_data)(6, './data/a.csv')
bob(gen_data)(10, './data/b.csv')
carol(gen_data)(11, './data/c.csv')

可以看到上面输出了一个 `PYUObject`，他是最后一行（也就是 carol）执行的返回值。不管实际执行的函数有没有返回值，交给 `PYU` 执行之后都会返回 `PYUObject`。这里返回的 `PYUObject` 实际值是 `None`，它虽然没有内容，但是可以用于 `sf.wait` 来等待函数执行完成，有时候是很有用的。

我们来看一下生成的3份数据，后面我们将使用他们来完成计算均值的任务：

In [7]:
%%!
cat ./data/a.csv

['6', '7', '8', '9', '10']

In [8]:
%%!
cat ./data/b.csv

['10', '11', '12', '13', '14', '15', '16', '17', '18']

In [9]:
%%!
cat ./data/c.csv

['11', '12', '13', '14', '15', '16', '17', '18', '19', '20']

### 加载数据

In [10]:
def load_data(file_name):
    data = np.loadtxt(file_name, dtype=np.int32)
    return data


alice_data = alice(load_data)('./data/a.csv')
bob_data = bob(load_data)('./data/b.csv')
carol_data = carol(load_data)('./data/c.csv')
alice_data, bob_data, carol_data

(<secretflow.device.device.pyu.PYUObject at 0x7f4a9b8c08b0>,
 <secretflow.device.device.pyu.PYUObject at 0x7f4bfef4b2b0>)

上面代码已经把数据加载到了各方内存中，我们尝试着看一下数据`alice_data, bob_data, carol_data`，可以看到他们是一个`secretflow.device.device.pyu.PYUObject`的实例。

`PYUObject` 代表了对一个 PYU 设备上的数据的引用，它本身不包含数据，我们尝试看一下`alice_data`的内容，可以看到它包含一个引用 `ObjectRef` 和他所在的 PYU `PYURuntime(alice)`(不同的组网模式会有一些不同)。

In [11]:
from pprint import pprint

pprint(vars(alice_data))

{'data': ObjectRef(f4402ec78d3a2607ffffffffffffffffffffffff0100000001000000),
 'device': PYURuntime(alice)}


这里需要注意两点：
1. 逻辑上，一个 `PYUObject` 只能被它所在的 `PYU` 访问和处理，其他逻辑设备无法访问；
2. 物理上，一个 `PYUObject` 只存在于它所在的 `PYU` 对应的物理设备的内存中，其他物理设备无法访问（生产模式下）。

在开发过程中，我们可以用 `sf.reveal`（也就是把数据公开出来）的方式拿到真实的数据，然后打印出来。

**特别注意：在生产代码中 `sf.reveal` 几乎不应该被使用，只有在算法中明确需要公开的数据才应该使用它**

In [12]:
print(sf.reveal(alice_data))

[ 6  7  8  9 10]


但在线上运行的时候我们不可能直接公开数据，但是又有打印中间数据的需求。我们仍然是可以做到这一点的，但是只能在数据所在的 `PYU` 上打印。

执行下面的代码，可以看到内容是在一个其他进程中打印出来的，在仿真模式中，这个进程代表 alice 执行了`print_data`函数；如果是在生产模式，那么只有真正的 alice 方可以把内容打印出来。

In [13]:
def print_data(msg, data):
    print(f'{msg}: {data}')


_ = alice(print_data)('alice_data', alice_data)

(_run pid=3908759) alice_data: [ 6  7  8  9 10]


### 计算均值

#### 设计算法

现在我们的三方各自有一些数据，需要算一个整体的均值，为了一定程度上保护原始数据，我们可以每方各自求和，并把求和结果和数据量发送给 dave，然后 dave 计算均值：$$avg=\frac{sum(a)+sum(b)+sum(c)}{n_a+n_b+n_c}$$

#### 算法实现

首先三方各自求和并统计数据量，我们来分别展示一下基于 PYU 编程的几种模式。

方法一：定义并执行 Task

Task 是一个 PYU 的执行单元，他表示在 PYU 上执行一个 python 函数。语法如下所示，需要执行的函数可以是自定义函数，也可以是库函数或者内置函数，函数的参数放在第二个括号中，Task 返回的结果是一个 PYUObject。

需要注意，下面的 `Task 1` 和 `Task 2` 两行只是提交了两个 Task，并没有等待执行完成，Task 的计算是异步完成的。

In [14]:
def sum(data):
    return np.sum(data)


alice_sum = alice(sum)(alice_data)  # Task 1
alice_shape = alice(np.shape)(alice_data)  # Task 2

# 看一下 Task 的返回值
print(alice_sum, alice_shape)

<secretflow.device.device.pyu.PYUObject object at 0x7f4bfef4ba00> <secretflow.device.device.pyu.PYUObject object at 0x7f4bfefc9280>


我们用 reveal 看一下结果：

In [15]:
# wait 可以等待 PYUObject 对应的 Task 执行完成
sf.wait([alice_sum, alice_shape])

# reveal 也有等待完成的效果，但同时会把数据公开出来
print(f'alice sum={sf.reveal(alice_sum)}, shape={sf.reveal(alice_shape)}')

alice sum=40, shape=(5,)


方法二：Task 多返回值，函数可以有多返回值，可以用指定 num_returns 的方式让 Task 返回多个 PYUObject。如果不指定 num_returns，那么框架会尝试自动推导应该返回多少个 PYUObject，如果推导失败所有返回值会被封装成一个 PYUObject。除非函数特别简单，否则我们还是建议明确指定 num_returns。

In [16]:
def agg(data):
    return np.sum(data), np.shape(data)


bob_sum, bob_shape = bob(agg, num_returns=2)(bob_data)

print(bob_sum, bob_shape)

<secretflow.device.device.pyu.PYUObject object at 0x7f4bfef48790> <secretflow.device.device.pyu.PYUObject object at 0x7f4c2453be20>


In [17]:
print(f'bob sum={sf.reveal(bob_sum)}, shape={sf.reveal(bob_shape)}')

bob sum=126, shape=(9,)


方法三：使用 Actor 管理数据和计算。

Actor 可以认为是在特定 device 上的一个类的实例，除了 `__init__` 以外，其他成员函数调用都会被封装成一个 Task，意味着成员函数的返回值会变成 PYUObject，参数也可以接受 PYUObject 或者普通变量。

值得注意的是：
1. Actor 会常驻在一个特定进程中，因此 Actor 内部可以保存状态；
2. Actor 成员函数的参数和返回值必须是可序列化的（普通 Task 也一样），但是其内部状态没有这个要求，可以是任意数据；
3. Actor 的 `__init__` 比较特殊，他会给原始类的 `__init__` 增加一个 `device` 参数，用来表示这个 Actor 实例在哪个 PYU 设备上;
4. Actor 的 `__init__` 不能接收 PYUObject 作为参数，只能接收基本参数（全局共享的参数），但是 `__init__` 函数内可以正常访问 Actor 的内部状态。

In [18]:
from secretflow import proxy, PYUObject


@proxy(PYUObject)
class Stat(object):
    def __init__(self, file_name):
        self.file_name = file_name

    def load_data(self):
        self.data = np.loadtxt(self.file_name, dtype=np.int32)

    def set_data(self, data):
        self.data = data

    def sum(self):
        return np.sum(self.data)

    def shape(self):
        return np.shape(self.data)


carol_actor = Stat('./data/c.csv', device=carol)

INFO:root:Create proxy actor <class '__main__.Stat'> with party carol.


上面我们定义了一个叫做 `Stat` 的 Actor 类，并且初始化了一个实例 `carol_actor`，我们初始化时指定了 `device=carol`，所以这个实例会在 `carol` 上运行。

接下来我们就可以像普通对象一样使用 `carol_actor` 了，但是要记住，他们返回值都是 `PYUObject`：

In [19]:
carol_actor.load_data()
# 或者直接传入已读取的数据：
# carol_actor.set_data(carol_data)
carol_sum = carol_actor.sum()
carol_shape = carol_actor.shape()

print(f'carol sum={sf.reveal(carol_sum)}, shape={sf.reveal(carol_shape)}')

carol sum=155, shape=(10,)


上面三方各自的计算已经完成，接下来需要把这些部分的结果汇总到 dave 上，然后在 dave 上计算出最终结果。

接下来我们把数据传递到 dave 上：

In [20]:
alice_sum_d = alice_sum.to(dave)
alice_shape_d = alice_shape.to(dave)

bob_sum_d = bob_sum.to(dave)
bob_shape_d = bob_shape.to(dave)

carol_sum_d = carol_sum.to(dave)
carol_shape_d = carol_shape.to(dave)

alice_sum_d, alice_shape_d

(<secretflow.device.device.pyu.PYUObject at 0x7f4bfef4c9a0>,
 <secretflow.device.device.pyu.PYUObject at 0x7f4bfef4cdf0>)

然后计算均值，下面展示了另外一个小技巧：作为 Task 的参数时，PYUObject 可以放到 list 或 dict 等结构中，并且不限制层次（比如可以是 dict 中的 list 的元素）。

In [21]:
def calc_avg(sums, shapes):
    total_sum = np.sum(sums)
    total_shape = np.sum(shapes)
    return total_sum / total_shape


dave_sums = [alice_sum_d, bob_sum_d, carol_sum_d]
dave_shapes = [alice_shape_d, bob_shape_d, carol_shape_d]

dave_avg = dave(calc_avg)(dave_sums, dave_shapes)

public_avg = sf.reveal(dave_avg)
print(f'avg={sf.reveal(public_avg)}, type={type(public_avg)}')

avg=13.375, type=<class 'numpy.float64'>


最后本地计算一下，可以看到计算结果是一样的。

In [22]:
data_a = np.loadtxt('./data/a.csv', dtype=np.int32)
data_b = np.loadtxt('./data/b.csv', dtype=np.int32)
data_c = np.loadtxt('./data/c.csv', dtype=np.int32)
data = np.concatenate([data_a, data_b, data_c])

avg = np.sum(data) / np.shape(data)
print(f'avg={avg}, type={type(avg)}')

avg=[13.375], type=<class 'numpy.ndarray'>


上面的本地计算中，故意使用了 `np.sum(data)/np.shape(data)` 计算均值，可以看到本地计算结果和基于 PYU 的计算结果并不完全一致：他们的类型不同。

原因在于 python 的特点和 num_returns 参数：在 python 中一个函数有多个返回值时，实际上返回的是一个 tuple, 因此下面两个函数是完全等价的。同样对于 num_returns 参数来说，下面两个函数都可以设置为 `num_returns=2`。

In [23]:
def multi_ret():
    return 1, 2


def multi_ret2():
    return (1, 2)

当返回值真的有多个时一切都很好，但是如果返回值只有一个，并且返回值是 array-like 的（tuple、list 等），且返回的 array 中只有一个值时（如下面的情况），PYU 就无法区分应该返回 array 中的元素还是 array 本身了，这种情况下，PYU 都会返回 array 中的唯一的那个元素值（也就是会解包），虽然这不一定是编程者的意图。

In [24]:
def single_ret():
    return 1


def single_ret2():
    return (1,)

**这是一个需要被着重注意的细节：如果一个函数的返回值是一个不确定长度的 list，当返回值的元素个数为 1 的时候，它会返回了 list 内的元素本身。不做特殊处理的情况下容易出现意想不到的 bug**

我们再来看一下计算过程：

In [25]:
def calc_avg(sums, shapes):
    total_sum = np.sum(sums)
    total_shape = np.sum(shapes)
    # 此处返回了一个长度为 1 的 array-like 数据
    return total_sum / total_shape  # [x]


dave_sums = [alice_sum_d, bob_sum_d, carol_sum_d]
dave_shapes = [alice_shape_d, bob_shape_d, carol_shape_d]

# 此处不填 num_returns 等价于 num_returns=1，此时框架会按照 num_returns 的设定，把长度为 1 的 array-like 的返回值解包
dave_avg = dave(calc_avg, num_returns=1)(dave_sums, dave_shapes)

public_avg = sf.reveal(dave_avg)
print(f'avg={sf.reveal(public_avg)}, type={type(public_avg)}')

avg=13.375, type=<class 'numpy.float64'>


## 混合使用 SPU 密态设备编程

在这上节我们展示了 PYU 明文设备的用法和一些注意事项，并且展示了如何仅使用 PYU 做多方计算。但是仅仅使用 PYU 做多方计算也许不够安全，我们回顾一下公式：$$avg=\frac{sum(a)+sum(b)+sum(c)}{n_a+n_b+n_c}$$

在上述例子中，虽然每方都只向 dave 透露统计值，但是统计值仍然包含一些信息量：dave 知道了每一方的总和和数据量。这些信息是否“有害”就见仁见智了。

那么如何让计算过程中的信息泄露最小化甚至完全消除呢？各类密文设备就是一种选择。


我们仍然使用上面的例子，但是希望用 SPU 密态设备保护中间计算过程，消除中间信息泄露。

与 PYU 不同，SPU 并不对应一个物理设备，而是由一组物理设备组成的一个虚拟的逻辑设备。SPU 的基础知识建议阅读[这篇文档](../developer/design/spu.rst)

根据底层协议的不同，SPU 需要不同数量的物理设备组成，在这个例子中，我们选择需要 3 方参与的 ABY3 协议，并且选择 alice、bob、carol 三方组成 SPU。

In [26]:
import spu
from secretflow.utils.testing import unused_tcp_port

aby3_cluster_def = {
    'nodes': [
        {'party': 'alice', 'address': f'127.0.0.1:{unused_tcp_port()}'},
        {'party': 'bob', 'address': f'127.0.0.1:{unused_tcp_port()}'},
        {'party': 'carol', 'address': f'127.0.0.1:{unused_tcp_port()}'},
    ],
    'runtime_config': {
        'protocol': spu.spu_pb2.ABY3,
        'field': spu.spu_pb2.FM64,
    },
}

my_spu = sf.SPU(aby3_cluster_def)

(SPURuntime pid=3929327) 2023-11-06 20:31:07.369 [info] [default_brpc_retry_policy.cc:DoRetry:52] socket error, sleep=1000000us and retry
(SPURuntime pid=3929324) 2023-11-06 20:31:07.438 [info] [default_brpc_retry_policy.cc:DoRetry:52] socket error, sleep=1000000us and retry
(SPURuntime pid=3929327) 2023-11-06 20:31:08.369 [info] [default_brpc_retry_policy.cc:LogHttpDetail:29] cntl ErrorCode '112', http status code '200', response header '', error msg '[E111]Fail to connect Socket{id=0 addr=127.0.0.1:54823} (0x0x3cbee40): Connection refused [R1][E112]Not connected to 127.0.0.1:54823 yet, server_id=0'
(SPURuntime pid=3929327) 2023-11-06 20:31:08.369 [info] [default_brpc_retry_policy.cc:DoRetry:75] aggressive retry, sleep=1000000us and retry
(SPURuntime pid=3929324) 2023-11-06 20:31:08.439 [info] [default_brpc_retry_policy.cc:LogHttpDetail:29] cntl ErrorCode '112', http status code '200', response header '', error msg '[E111]Fail to connect Socket{id=0 addr=127.0.0.1:58731} (0x0x36f6780)

我们仍然使用 PYU 加载本地数据：

In [27]:
def agg(data):
    return np.sum(data), np.shape(data)


alice_sum, alice_shape = alice(agg, num_returns=2)(alice_data)
bob_sum, bob_shape = bob(agg, num_returns=2)(bob_data)
carol_sum, carol_shape = carol(agg, num_returns=2)(carol_data)

目前 3 个 sum 和 3 个 shape 都分别在各自本地，因此是安全的。

接下来需要把这些数据“放”到 `my_spu` 中计算均值，如果你了解一些[秘密分享](../developer/design/spu.rst#mpc-secure-multi-party-computation)协议就会知道，这一步骤需要各方本地做数据分片，并把部分分片发送到协议的其他方（也就是组成 SPU 的其他参与方）。

这个步骤是相对复杂的，但是 SPU 设备在编程上屏蔽了这种复杂性：把数据从 PYU “放”到 SPU 就和在 PYU 之间传递一样简单。

In [28]:
alice_sum_spu = alice_sum.to(my_spu)
bob_sum_spu = bob_sum.to(my_spu)
carol_sum_spu = carol_sum.to(my_spu)

alice_shape_spu = alice_shape.to(my_spu)
bob_shape_spu = bob_shape.to(my_spu)
carol_shape_spu = carol_shape.to(my_spu)

(_run pid=3909057) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=3909057) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=3909057) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
(_run pid=3909057) INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
(_run pid=3909057) WARNING:jax._src.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=3909321) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=39

我们来看一下放到 SPU 以后，数据会变成什么样。

跟 PYU 类似，把数据放到 SPU 以后，会得到一个 `secretflow.device.device.spu.SPUObject`, 这个 `SPUObject` 也是一个 SPU 中的数据的引用。和 PYU 不同，`SPUObject` 并不引用一个物理的数据，而是表示一个多方组成的逻辑数据，我们无法看到这个逻辑数据的内容，但是可以让这个逻辑数据在 `my_spu` 上执行计算。

In [29]:
alice_sum_spu, alice_shape_spu

(<secretflow.device.device.spu.SPUObject at 0x7f4bfef4c700>,
 <secretflow.device.device.spu.SPUObject at 0x7f4bfef07be0>)

同样来看一下 `SPUObject` 的内容。一个 `SPUObject` 中同样包含在所在的 spu 设备信息，此外还包含一个表示数据“形状”的 meta 信息和分片信息（不同组网模式下会有一些不同）。

In [30]:
from pprint import pprint

pprint(vars(alice_sum_spu))

{'device': <secretflow.device.device.spu.SPU object at 0x7f4bfefc9a00>,
 'meta': ObjectRef(88543757a8df6d2fffffffffffffffffffffffff0100000001000000),
 'shares_name': [ObjectRef(347cc60e0bb3da749fce953dc1bbfc1af4f50ef60100000001000000),
                 ObjectRef(a02c24b8b7fc0a314bfe84a2ddef3fc302eba3980100000001000000),
                 ObjectRef(a631fe8d231813bfab4501800d1eaed6ccbc32510100000001000000)]}


现在所有数据都放到 `my_spu` 中，我们可以开始最终的均值计算了。整个计算都在密态进行，我们只将最终的结果公开出来。 

In [31]:
import jax.numpy as jnp


def avg(sums, shapes):
    sums = jnp.array(sums)
    shapes = jnp.array(shapes)
    res = jnp.sum(sums) / jnp.sum(shapes)
    return res


spu_sums = [alice_sum_spu, bob_sum_spu, carol_sum_spu]
spu_shapes = [alice_shape_spu, bob_shape_spu, carol_shape_spu]

spu_avg = my_spu(avg)(spu_sums, spu_shapes)

public_avg = sf.reveal(spu_avg)
print(f'avg={public_avg}, type={type(public_avg)}')

avg=13.374797821044922, type=<class 'numpy.ndarray'>


我们关注一下几个“细节”：
1. SPU 执行的函数是使用 `jax.numpy` 写的，这个跟 SPU 的[设计](https://www.secretflow.org.cn/docs/spu/latest/en-US/getting_started/introduction)有关，目前只支持 JAX，支持的 JAX 算子的完整列表可以查看[文档](https://www.secretflow.org.cn/docs/spu/latest/en-US/reference/np_op_status)。

2. 可以看到 SPU 的计算结果是正确的，但是精度不太高，这是因为我们用到了比较复杂的“除法”操作，可以考虑初始化 SPU 时更精细的设置 RuntimeConfig，
RuntimeConfig 的完整选项可以查看 [RuntimeConfig 文档](https://www.secretflow.org.cn/docs/spu/latest/en-US/reference/runtime_config#runtimeconfig)


接下来我们调整一下参数，再构造一个新的 spu 实例，看看能不能把计算精度提高：

In [32]:
import secretflow as sf
import spu
from secretflow.utils.testing import unused_tcp_port

aby3_cluster_def2 = {
    'nodes': [
        {'party': 'alice', 'address': f'127.0.0.1:{unused_tcp_port()}'},
        {'party': 'bob', 'address': f'127.0.0.1:{unused_tcp_port()}'},
        {'party': 'carol', 'address': f'127.0.0.1:{unused_tcp_port()}'},
    ],
    'runtime_config': {
        'protocol': spu.spu_pb2.ABY3,
        'field': spu.spu_pb2.FM64,
        'fxp_fraction_bits': 24,
        'fxp_div_goldschmidt_iters': 6,
    },
}

my_spu2 = sf.SPU(aby3_cluster_def2)

(SPURuntime pid=3934768) 2023-11-06 20:37:21.329 [info] [default_brpc_retry_policy.cc:DoRetry:52] socket error, sleep=1000000us and retry
(SPURuntime pid=3934768) 2023-11-06 20:37:22.329 [info] [default_brpc_retry_policy.cc:LogHttpDetail:29] cntl ErrorCode '112', http status code '200', response header '', error msg '[E111]Fail to connect Socket{id=0 addr=127.0.0.1:35155} (0x0x343db00): Connection refused [R1][E112]Not connected to 127.0.0.1:35155 yet, server_id=0'
(SPURuntime pid=3934768) 2023-11-06 20:37:22.330 [info] [default_brpc_retry_policy.cc:DoRetry:75] aggressive retry, sleep=1000000us and retry
(SPURuntime pid=3934768) 2023-11-06 20:37:23.330 [info] [default_brpc_retry_policy.cc:LogHttpDetail:29] cntl ErrorCode '112', http status code '200', response header '', error msg '[E111]Fail to connect Socket{id=0 addr=127.0.0.1:35155} (0x0x343db00): Connection refused [R1][E112]Not connected to 127.0.0.1:35155 yet, server_id=0 [R2][E112]Not connected to 127.0.0.1:35155 yet, server_id

In [33]:
# 把数据放到新的 spu 实例上
alice_sum_spu = alice_sum.to(my_spu2)
bob_sum_spu = bob_sum.to(my_spu2)
carol_sum_spu = carol_sum.to(my_spu2)

alice_shape_spu = alice_shape.to(my_spu2)
bob_shape_spu = bob_shape.to(my_spu2)
carol_shape_spu = carol_shape.to(my_spu2)

在 `my_spu2` 上重新计算一次：

In [34]:
spu_sums = [alice_sum_spu, bob_sum_spu, carol_sum_spu]
spu_shapes = [alice_shape_spu, bob_shape_spu, carol_shape_spu]

spu_avg = my_spu2(avg)(spu_sums, spu_shapes)

public_avg = sf.reveal(spu_avg)
print(f'avg={public_avg}, type={type(public_avg)}')

avg=13.375, type=<class 'numpy.ndarray'>


当然除了通过 `reveal` 的方式公开结果，也可以把结果给到特定的“结果方”。SPU 到 PYU 的传递相当于把秘密分享分片集中到 PYU 中，重构出真实的数据，这个过程在编程中也非常简单：

In [35]:
dave_avg = spu_avg.to(dave)

dave_avg

到这里结果数据已经传递到了 dave，dave 可以进行本地计算或者保存等等操作。我们把 dave 的结果 `reveal` 出来看一下是否正确。

In [36]:
print(f'dave_avg={sf.reveal(dave_avg)}')

dave_avg=13.375


### 算法封装

上面介绍了如何明密文混合编程，更多时候我们需要写一些封装好的算法或者功能模块，这里给出一些建议：
1. 尽量不要在算法或功能模块内初始化集群和逻辑设备，把逻辑设备作为参数传入，可以让算法更通用；
2. 有时候在算法中 PYU 明文设备和 SPU 密文设备是可替代的，对于这些部分建议写一套代码来支持（即用 jax 来写计算逻辑）。

接下来我们把三方求均值做成一个可复用的功能模块。

In [37]:
from typing import Dict
from secretflow.device import Device


def avg(files: Dict[Device, str], agg_device: Device, result_device: Device):
    '''
    参数：
        files: PYU -> filename，表示每一方的文件路径
        agg_device: 表示最终计算均值的虚拟设备，可以是 PYU 或者 SPU
        result_device：表示获取结果的虚拟设备，一般来说是 PYU
    '''

    def _load_data(file_name):
        return np.loadtxt(file_name, dtype=np.int32)

    data = {device: device(_load_data)(fname) for device, fname in files.items()}

    def _stat(data):
        return np.sum(data), len(data)

    sums = []
    lengths = []
    for device, datum in data.items():
        sum, length = device(_stat, num_returns=2)(datum)
        sums.append(sum.to(agg_device))
        lengths.append(length.to(agg_device))

    def _agg(sums, lengths):
        sums = jnp.array(sums)
        lengths = jnp.array(lengths)
        return jnp.sum(sums) / jnp.sum(lengths)

    result = agg_device(_agg)(sums, lengths)
    return result.to(result_device)

试着执行一下，先用明文设备 dave 作为聚合设备，并把最终结果传递给 alice

In [38]:
files = {
    alice: './data/a.csv',
    bob: './data/b.csv',
    carol: './data/c.csv',
}

result = avg(files=files, agg_device=dave, result_device=alice)

alice(print_data)('result', result)

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.


(_run pid=3909321) result: 13.375


这次用 spu 作为聚合的逻辑设备，同样让结果在 alice 重建出来 

In [39]:
# 用低精度的 SPU 做聚合
result = avg(files=files, agg_device=my_spu, result_device=alice)
alice(print_data)('my_spu', result)

# 用高精度的 SPU 做聚合
result = avg(files=files, agg_device=my_spu2, result_device=alice)
alice(print_data)('my_spu2', result)

(_run pid=3909321) my_spu: 13.374794006347656


(_run pid=3909321) my_spu2: 13.375


## 更多密态设备

### HEU 同态加密设备

HEU 是隐语的另一个密态设备。HEU 的使用方式与 SPU 或 PYU 差异较大，HEU 目前不支持可编程能力，即：不能定义一个函数然后交给 HEU 执行。HEU 目前更像是一个同态加密库，SecretFlow 对他做了一些封装，使得它可以和其他逻辑设备联合使用。


HEU 的初始化也相对复杂，需要对同态加密有一定的了解，建议先阅读 [HEU 文档](https://www.secretflow.org.cn/docs/heu/latest/zh-Hans/getting_started/quick_start)了解基础概念。在编程中需要特别注意 `sk_keeper` 的设置，他拥有私钥可以解密数据，因此大多数时候加密数据不应该交给 `sk_keeper` 来计算。

In [40]:
import spu

my_heu = sf.HEU(
    {
        'sk_keeper': {'party': 'alice'},
        'evaluators': [{'party': 'bob'}, {'party': 'carol'}, {'party': 'dave'}],
        'mode': 'PHEU',
        'he_parameters': {
            'schema': 'paillier',
            'key_pair': {'generate': {'bit_size': 2048}},
        },
    },
    spu.spu_pb2.FM128,
)

数据转移到 HEU 时需要一个 `HEUMoveConfig` ，它包含两个参数：
1. heu_encoder： 用来指定数据编码方式，根据原始数据的类型选择用整型编码器或者浮点数编码器；
2. heu_dest_party：表示数据实际传输到哪一方，HEU 会自动决定是否需要加密：如果 `heu_dest_party` 与 原始数据所在的 `PYU` 一致就**不会加密**数据，如果数据出现了跨域传输 HEU 就会把它**加密**。

在编程过程中，有时候需要注意 `HEUObject` 是明文还是密文状态，对于加法同态来说，乘法只能是一个密文乘以一个明文（可以理解为多个密文的和）。

In [41]:
from secretflow.device.device.heu import HEUMoveConfig
from heu import phe

move_config = HEUMoveConfig(
    heu_dest_party='dave',
    heu_encoder=phe.IntegerEncoder(my_heu.schema),
)

定义好 `move_config` 后，我们就可以用类似其他逻辑设备的方式把数据放到 `HEU` 了：

In [42]:
alice_sum_heu = alice_sum.to(my_heu, move_config)
bob_sum_heu = bob_sum.to(my_heu, move_config)
carol_sum_heu = carol_sum.to(my_heu, move_config)

alice_shape_heu = alice_shape.to(my_heu, move_config)
bob_shape_heu = bob_shape.to(my_heu, move_config)
carol_shape_heu = carol_shape.to(my_heu, move_config)

同样看一下 `HEU` 里面的数据是什么样的，因为我们的数据进行了跨设备移动，所以得到的 HEUObject 是加密状态的：

In [43]:
pprint(vars(alice_sum_heu))

{'data': ObjectRef(f0133118a11aef4156f60ab8810d57c91a9063380100000001000000),
 'device': <secretflow.device.device.heu.HEU object at 0x7f4bfef18b20>,
 'is_plain': False,
 'location': 'dave'}


与其他逻辑设备不同，可以直接在 `HEUObject` 执行运算：

In [44]:
total_sum_heu = alice_sum_heu + bob_sum_heu + carol_sum_heu
total_shape_heu = alice_shape_heu + bob_shape_heu + carol_shape_heu

计算的结果仍然是一个 `HEUObject`：

In [45]:
pprint(vars(total_sum_heu))

{'data': ObjectRef(7ef10a5289b2683b2b83696828bdba01a38857140100000001000000),
 'device': <secretflow.device.device.heu.HEU object at 0x7f4bfef18b20>,
 'is_plain': False,
 'location': 'dave'}


HEU 目前不支持除法，所以需要把除法操作交给其他逻辑设备。

数据在 `HEU` 到其他设备的传递也使用同样的 API，但是根据目标设备的不同(`PYU` 或者 `SPU`)，背后的实现方式非常不同。在编程的过程中你只需要知道：数据在 `SPU` 和 `HEU` 之间传递是安全的，中间没有任何数据信息泄露。

下面的例子中，我们把数据放到 PYU 设备上执行除法操作：

In [46]:
total_sum_alice = total_sum_heu.to(alice)
total_shape_alice = total_shape_heu.to(alice)

total_sum_alice, total_shape_alice

(<secretflow.device.device.pyu.PYUObject at 0x7f4bfefc9310>,
 <secretflow.device.device.pyu.PYUObject at 0x7f4bfefc9220>)

In [47]:
def calc_avg(total_sum, total_shape):
    return total_sum / total_shape


avg_alice = alice(calc_avg)(total_sum_alice, total_shape_alice)

print(f"avg_alice={sf.reveal(avg_alice)}")

avg_alice=[13.375]


HEUObject 支持加法、减法、乘法和矩阵乘法（+, -, *, @）几种运算，加减法的两个操作数都可以是明文或密文，但是乘法的两个操作数必须有一个是明文。因为包括我们上面选用的 paillier 在内，HEU 目前只支持同态加法，详细算法列表可以查看 [HEU 文档](https://www.secretflow.org.cn/docs/heu/latest/zh-Hans/getting_started/algo_choice)。

### TEEU 可信执行环境逻辑设备

可信执行环境(Trusted Execution Environment，TEE) 是一种基于硬件的隐私保护技术。它保证了执行代码的真实性，运行时状态(如 CPU 寄存器、内存和敏感 I/O)的完整性，以及存储在内存中的代码、数据和运行时状态的机密性。此外，还应能够向第三方提供远程认证，以证明其可靠性。

TEEU(TEE processing Unit)是 SecretFlow 中的 TEE 逻辑设备，通过 TEEU，用户可以方便的把数据放在TEE内进行计算，并且达到保护数据完整和安全的目的。

TEEU 的编码体验与 PYU 类似，只要注意 `to` 的 `allow_funcs` 参数，表示允许这个数据在 `TEEU` 中执行的逻辑。但是由于 TEE 的特点，TEEU 的部署相对复杂，建议仔细阅读[这篇文档](https://www.secretflow.org.cn/docs/secretflow/latest/zh-Hans/tutorial/teeu)。

示例代码：
```python
from secretflow.device import TEEU

def average(data):
    return np.average(data, axis=1)

# mrenclave can be omitted in simulation mode.
teeu = TEEU('carol', mr_enclave='')

# Transfer data to teeu.
a_teeu = alice_data.to(teeu, allow_funcs=average)
b_teeu = bob_data.to(teeu, allow_funcs=average)

# TEEU runs average.
avg_val = teeu(average)([a_teeu, b_teeu])
```